In [245]:
import pandas as pd
pd.set_option('display.max_columns', 30)

In [246]:
weekly = pd.read_parquet('data/weekly.parquet')

In [247]:
depth = pd.read_csv('data/depth.csv', index_col=0).sort_values(by='depth_team')
depth['dense_depth'] = depth.groupby(['club_code','season','week','position', 'game_type','formation']).cumcount()
off = depth.loc[depth.formation=='Offense']

In [248]:
context_cols = ['player_id', 'player_display_name', 'position',
       'position_group',
]
game_context = ['week','season_type', 'season', 'opponent_team', 'recent_team', ]
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr']
rush_stat_cols = [
     'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'fantasy_points', 'fantasy_points_ppr'
]

depth_cols = ['season', 'last_name',
       'first_name', 'football_name', 'formation', 'gsis_id', 'jersey_number',
       'position', 'elias_id', 'depth_position', 'full_name']

In [249]:
#week_small = weekly.loc[(weekly.player_id=='00-0039746') & (weekly.season==2024)]
hmm = off.merge(
    weekly,
    how="outer",
    left_on=["gsis_id", "club_code", "season", "week", "position"],
    right_on=["player_id", "recent_team", "season", "week", "position"],
).sort_values(by="week").reset_index(drop=True)


In [270]:
agg.loc[(agg.full_name=='Saquon Barkley')
        & (agg.season==2024)][['week'] + context_cols + depth_cols]

,week,player_id,player_display_name,position,position_group,season,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name
60,1.0,00-0034844,Saquon Barkley,RB,RB,2024,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley
13981,2.0,00-0034844,Saquon Barkley,RB,RB,2024,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley
29574,3.0,00-0034844,Saquon Barkley,RB,RB,2024,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley
38198,4.0,00-0034844,Saquon Barkley,RB,RB,2024,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley
58865,5.0,00-0034844,Saquon Barkley,RB,RB,2024,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley
71074,6.0,00-0034844,Saquon Barkley,RB,RB,2024,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley
82293,7.0,00-0034844,Saquon Barkley,RB,RB,2024,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley
92198,8.0,00-0034844,Saquon Barkley,RB,RB,2024,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley
111644,9.0,00-0034844,Saquon Barkley,RB,RB,2024,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley
114951,10.0,00-0034844,Saquon Barkley,RB,RB,2024,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley


In [251]:
hmm[context_cols] = hmm.groupby(depth_cols)[context_cols].bfill()
hmm[context_cols] = hmm.groupby(depth_cols)[context_cols].ffill()
hmm[stat_cols] = hmm[stat_cols].fillna(0)

In [252]:
player_fantasy = hmm.copy(deep=True)  #[context_cols + game_context + rush_stat_cols]
player_fantasy.loc[player_fantasy.gsis_id=="00-0039746"]


,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,...,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
12132,2024,PHI,1.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17405,2024,PHI,2.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27197,2024,PHI,3.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50169,2024,PHI,4.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52807,2024,PHI,5.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73625,2024,PHI,6.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76623,2024,PHI,7.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,1.5
90002,2024,PHI,8.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.3,-0.3
104867,2024,PHI,9.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116754,2024,PHI,10.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4,3.4


In [253]:
player_fantasy[['ppr_roll', 'rush_roll', 'carries_roll']] = player_fantasy.groupby(context_cols)[['fantasy_points_ppr', 'rushing_yards', 'carries']].transform(lambda x: x.ewm(span=5, min_periods=0).mean())
player_fantasy.loc[player_fantasy.gsis_id=="00-0039746"]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,...,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,ppr_roll,rush_roll,carries_roll
12132,2024,PHI,1.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
17405,2024,PHI,2.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
27197,2024,PHI,3.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
50169,2024,PHI,4.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
52807,2024,PHI,5.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
73625,2024,PHI,6.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
76623,2024,PHI,7.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,1.5,0.531083,5.310831,2.832443
90002,2024,PHI,8.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.3,-0.3,0.242807,2.428073,2.543695
104867,2024,PHI,9.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159710,1.597100,1.673152
116754,2024,PHI,10.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4,3.4,1.258868,12.588677,4.158535


In [254]:
player_fantasy[context_cols + game_context + ['ppr_roll', 'rush_roll']].to_parquet('data/agg/player_weekly_agg.parquet')

In [255]:
player_fantasy.loc[
    (player_fantasy.recent_team=='PHI') &
    (player_fantasy.season==2024) & 
    (player_fantasy.week==11)
]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,...,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,ppr_roll,rush_roll,carries_roll
129009,2024,PHI,11.0,REG,1.0,Goedert,Dallas,Dallas,Offense,00-0034351,88.0,TE,GOE595520,TE,Dallas Goedert,...,32.0,29.0,2.0,3.805890,0.0,1.906250,0.20,0.198758,0.439130,0.0,6.100000,11.100000,6.426992,0.000000,0.000000
129797,2024,PHI,11.0,REG,1.0,Barkley,Saquon,Saquon,Offense,00-0034844,26.0,RB,BAR192674,RB,Saquon Barkley,...,9.0,47.0,1.0,3.788334,0.0,5.777778,0.12,0.055901,0.219130,0.0,31.799999,33.799999,20.792524,84.226585,16.266305
130060,2024,PHI,11.0,REG,1.0,Brown,Arthur,A.J.,Offense,00-0035676,11.0,WR,BRO413223,WR,A.J. Brown,...,73.0,22.0,4.0,1.713152,0.0,0.890411,0.32,0.453416,0.797391,0.0,6.500000,11.500000,6.707228,0.691269,0.098810
131844,2024,PHI,11.0,REG,1.0,Dotson,Jahan,Jahan,Offense,00-0037741,83.0,WR,DOT282798,WR,Jahan Dotson,...,1.0,14.0,0.0,-1.022546,0.0,8.000000,0.08,0.006211,0.124348,0.0,0.800000,1.800000,3.753644,0.673248,0.222732
132811,2024,PHI,11.0,REG,1.0,Smith,DeVonta,DeVonta,Offense,00-0036912,6.0,WR,SMI206016,WR,DeVonta Smith,...,43.0,18.0,2.0,-0.405444,0.0,0.674419,0.24,0.267081,0.546957,0.0,2.900000,6.900000,11.235466,0.000000,0.000000
132831,2024,PHI,11.0,REG,2.0,Gainwell,Kenneth,Kenneth,Offense,00-0036919,14.0,RB,GAI606964,RB,Kenneth Gainwell,...,3.0,3.0,0.0,-0.732195,0.0,2.000000,0.04,0.018634,0.073043,0.0,4.900000,5.900000,3.267742,18.408424,2.346379
136486,2024,PHI,11.0,REG,1.0,Hurts,Jalen,Jalen,Offense,00-0036389,1.0,QB,HUR767022,QB,Jalen Hurts,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0,18.740000,18.740000,22.623406,53.105503,11.916208


In [256]:
player_fantasy[['season', 'club_code', 'week', 'game_type', 'depth_team', 'last_name',
       'first_name', 'football_name', 'formation', 'gsis_id', 'jersey_number',
       'position', 'elias_id', 'depth_position', 'full_name', 'dense_depth']]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth
0,2010,PIT,1.0,REG,1.0,Adams,Flozell,Flozell,Offense,00-0000045,71.0,T,ADA280969,RT,Flozell Adams,1.0
1,2019,NE,1.0,REG,1.0,Wynn,Isaiah,Isaiah,Offense,00-0034833,77.0,G,WYN131632,LT,Isaiah Wynn,1.0
2,2011,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,NE,1.0,REG,1.0,Wynn,Isaiah,Isaiah,Offense,00-0034833,77.0,G,WYN131632,LT,Isaiah Wynn,2.0
4,2012,DEN,1.0,REG,2.0,Davis,Emmanuel,C.J.,Offense,00-0026705,66.0,G,DAV243165,C,C.J. Davis,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234821,2023,KC,NaN,SBBYE,2.0,Morris,Wanya,Wanya,Offense,00-0038981,64.0,T,MOR552993,LT,Wanya Morris,NaN
234822,2023,BAL,NaN,SBBYE,1.0,Flowers,Xavien,Zay,Offense,00-0039064,4.0,WR,FLO577002,WR,Zay Flowers,NaN
234823,2023,DET,NaN,SBBYE,1.0,LaPorta,Sam,Sam,Offense,00-0039065,87.0,TE,LAP640235,TE,Sam LaPorta,NaN
234824,2023,KC,NaN,SBBYE,1.0,Rice,Rashee,Rashee,Offense,00-0039067,4.0,WR,RIC152916,WR,Rashee Rice,NaN


In [257]:
agg = player_fantasy.copy()
agg = agg.sort_values(by=['club_code','season','week','full_name'])
agg.loc[(agg.full_name=='Will Shipley') & (agg.season==2024)]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,...,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,ppr_roll,rush_roll,carries_roll
12132,2024,PHI,1.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
17405,2024,PHI,2.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
27197,2024,PHI,3.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
50169,2024,PHI,4.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
52807,2024,PHI,5.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
73625,2024,PHI,6.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
76623,2024,PHI,7.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,1.5,0.531083,5.310831,2.832443
90002,2024,PHI,8.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.3,-0.3,0.242807,2.428073,2.543695
104867,2024,PHI,9.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159710,1.597100,1.673152
116754,2024,PHI,10.0,REG,3.0,Shipley,Will,Will,Offense,00-0039746,28.0,RB,SHI386967,RB,Will Shipley,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4,3.4,1.258868,12.588677,4.158535


In [258]:
def stack_players(df, num_players, index_cols, value_cols):
    'df should be seperated by position already'
    df = df.loc[df.dense_depth < num_players]
    piv = df.pivot(index=index_cols, columns=['position','dense_depth'], values=value_cols).reset_index()
    piv.columns = [x[0] + '_' + x[1] +'_' +  (str(int(x[2]))) if x[2] != '' else x[0] + x[1] for x in piv.columns ]
    return piv

In [259]:
agg.loc[agg.gsis_id == "00-0039746"][['week','carries','rush_roll','carries_roll']]

,week,carries,rush_roll,carries_roll
12132,1.0,0.0,0.000000,0.000000
17405,2.0,0.0,0.000000,0.000000
27197,3.0,0.0,0.000000,0.000000
50169,4.0,0.0,0.000000,0.000000
52807,5.0,0.0,0.000000,0.000000
73625,6.0,0.0,0.000000,0.000000
76623,7.0,8.0,5.310831,2.832443
90002,8.0,2.0,2.428073,2.543695
104867,9.0,0.0,1.597100,1.673152
116754,10.0,9.0,12.588677,4.158535


In [260]:
rb_stats = stack_players(agg.loc[(agg.position=='RB') & (~agg.week.isna())],3,  ['club_code','season','week', 'game_type'], ['gsis_id','full_name', 'ppr_roll','rush_roll', 'carries_roll'])
rb_stats = rb_stats.rename({'club_code':'team_name'},axis=1)
rb_stats[['ppr_roll_RB_2','rush_roll_RB_2', 'carries_roll_RB_2']] = rb_stats[['ppr_roll_RB_2','rush_roll_RB_2', 'carries_roll_RB_2']].fillna(0).astype(float)

C:\Users\Sharedw\AppData\Local\Temp\ipykernel_29460\634914049.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rb_stats[['ppr_roll_RB_2','rush_roll_RB_2', 'carries_roll_RB_2']] = rb_stats[['ppr_roll_RB_2','rush_roll_RB_2', 'carries_roll_RB_2']].fillna(0).astype(float)


In [261]:
rb_stats.sample(5)

,team_name,season,week,game_type,gsis_id_RB_1,gsis_id_RB_2,gsis_id_RB_0,full_name_RB_1,full_name_RB_2,full_name_RB_0,ppr_roll_RB_1,ppr_roll_RB_2,ppr_roll_RB_0,rush_roll_RB_1,rush_roll_RB_2,rush_roll_RB_0,carries_roll_RB_1,carries_roll_RB_2,carries_roll_RB_0
983,BUF,2018,10.0,REG,00-0027531,00-0031959,00-0027029,Chris Ivory,Marcus Murphy,LeSean McCoy,5.094613,2.657249,14.19201,25.864859,23.905345,70.118958,6.029416,4.790091,15.363315
1196,CAR,2014,14.0,REG,00-0026153,00-0029731,00-0024242,Jonathan Stewart,Fozzy Whittaker,DeAngelo Williams,14.522397,4.488777,3.538644,84.012254,4.726126,26.832353,11.827504,2.072279,6.779104
337,ATL,2013,10.0,REG,00-0026855,00-0028083,00-0022736,Antone Smith,Jacquizz Rodgers,Steven Jackson,1.23126,7.032447,7.517296,5.273474,20.090594,32.644034,1.889871,3.440151,10.832906
3059,GB,2011,11.0,REG,00-0027791,00-0028602,00-0022984,James Starks,Brandon Saine,Ryan Grant,10.050137,0.000000,1.779212,34.928429,0.000000,11.15092,12.04088,0.000000,4.347893
2048,CLE,2016,10.0,REG,00-0032257,00-0031158,00-0030656,Duke Johnson,George Atkinson,Isaiah Crowell,8.539082,0.000000,9.847571,13.794484,0.000000,36.708574,3.773379,0.000000,10.159251


In [262]:
rb_stats.loc[rb_stats.gsis_id_RB_2.isna()]

,team_name,season,week,game_type,gsis_id_RB_1,gsis_id_RB_2,gsis_id_RB_0,full_name_RB_1,full_name_RB_2,full_name_RB_0,ppr_roll_RB_1,ppr_roll_RB_2,ppr_roll_RB_0,rush_roll_RB_1,rush_roll_RB_2,rush_roll_RB_0,carries_roll_RB_1,carries_roll_RB_2,carries_roll_RB_0
210,ARI,2021,13.0,REG,00-0036383,NaN,00-0033553,Eno Benjamin,NaN,James Conner,1.68745,0.0,13.277737,4.916504,0.0,47.814148,2.16509,0.0,12.215301
211,ARI,2021,14.0,REG,00-0036383,NaN,00-0033553,Eno Benjamin,NaN,James Conner,0.522187,0.0,17.241281,1.678907,0.0,36.518844,1.30816,0.0,10.746232
230,ARI,2022,12.0,REG,00-0037299,NaN,00-0033553,Keaontay Ingram,NaN,James Conner,1.031632,0.0,9.083909,9.517672,0.0,48.367403,2.994435,0.0,10.608664
244,ARI,2023,7.0,REG,00-0038705,NaN,00-0037299,Emari Demercado,NaN,Keaontay Ingram,6.829617,0.0,1.256107,30.646103,0.0,7.709443,6.119719,0.0,2.255598
245,ARI,2023,8.0,REG,00-0038705,NaN,00-0037299,Emari Demercado,NaN,Keaontay Ingram,6.00079,0.0,4.031274,39.634174,0.0,9.541714,9.391523,0.0,3.67239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8633,WAS,2011,9.0,REG,00-0028043,NaN,00-0026279,Roy Helu,NaN,Ryan Torain,13.199069,0.0,3.428964,25.849624,0.0,12.651409,5.957882,0.0,4.407387
8642,WAS,2011,18.0,REG,00-0028115,NaN,00-0028043,Evan Royster,NaN,Roy Helu,6.896641,0.0,3.680815,41.540616,0.0,7.216413,7.082708,0.0,1.784647
8746,WAS,2017,12.0,REG,00-0033526,NaN,00-0030404,Samaje Perine,NaN,Chris Thompson,12.947248,0.0,1.896817,58.678349,0.0,4.475651,13.236567,0.0,1.222276
8826,WAS,2021,18.0,REG,00-0032975,NaN,00-0036755,Jonathan Williams,NaN,Jaret Patterson,2.297082,0.0,8.153862,19.431551,0.0,29.392966,4.141283,0.0,7.000888


In [263]:
rb_stats.to_csv('data/rb_stats.csv')

In [264]:
rb_stats.dtypes

team_name             object
season                 int64
week                 float64
game_type             object
gsis_id_RB_1          object
gsis_id_RB_2          object
gsis_id_RB_0          object
full_name_RB_1        object
full_name_RB_2        object
full_name_RB_0        object
ppr_roll_RB_1         object
ppr_roll_RB_2        float64
ppr_roll_RB_0         object
rush_roll_RB_1        object
rush_roll_RB_2       float64
rush_roll_RB_0        object
carries_roll_RB_1     object
carries_roll_RB_2    float64
carries_roll_RB_0     object
dtype: object